In [ ]:
# import sys
# import os

# sys.dont_write_bytecode = True
# os.environ["NUMBA_DISABLE_JIT"] = "1"

In [ ]:
import numpy as np
from my_stuff import MufexKeys
from quantfreedom.exchanges.mufex_exchange.mufex import Mufex
from quantfreedom.enums import *
from numba import njit, typed, types
from quantfreedom.nb_funcs.nb_custom_logger import *
from quantfreedom.nb_funcs.nb_helper_funcs import nb_float_to_str
from quantfreedom.helper_funcs import dos_cart_product
from quantfreedom.nb_funcs.nb_helper_funcs import nb_get_dos
from quantfreedom.nb_funcs.nb_order_handler.nb_stop_loss import *



mufex_main = Mufex(
    api_key=MufexKeys.api_key,
    secret_key=MufexKeys.secret_key,
    use_test_net=False,
)

%load_ext autoreload
%autoreload 2

In [ ]:
candles = mufex_main.get_candles(
    symbol="BTCUSDT",
    timeframe="5m",
    candles_to_dl=200,
)

In [ ]:
mufex_main.set_exchange_settings(
    symbol="BTCUSDT",
    position_mode=PositionModeType.HedgeMode,
    leverage_mode=LeverageModeType.Isolated,
)

# backtest_settings = BacktestSettings(
#     array_size=1000,
#     gains_pct_filter=0,
#     total_trade_filter=40,
#     upside_filter=0,
# )

backtest_settings = BacktestSettings()
dos_arrays = DynamicOrderSettingsArrays(
    max_equity_risk_pct=np.array([12]),
    max_trades=np.array([5]),
    risk_account_pct_size=np.array([3]),
    risk_reward=np.array([2.5]),
    sl_based_on_add_pct=np.array([0.05, 0.1, 0.15]),
    sl_based_on_lookback=np.array([20]),
    sl_bcb_type=np.array([CandleBodyType.Low]),
    sl_to_be_cb_type=np.array([CandleBodyType.Nothing]),
    sl_to_be_when_pct=np.array([0]),
    trail_sl_bcb_type=np.array([CandleBodyType.Low]),
    trail_sl_by_pct=np.array([0.5]),
    trail_sl_when_pct=np.array([1]),
)
static_os = StaticOrderSettings(
    increase_position_type=IncreasePositionType.RiskPctAccountEntrySize,
    leverage_strategy_type=LeverageStrategyType.Dynamic,
    logger_bool=True,
    num_candles=50,
    pg_min_max_sl_bcb="min",
    sl_strategy_type=StopLossStrategyType.SLBasedOnCandleBody,
    sl_to_be_bool=False,
    starting_bar=50,
    starting_equity=1000.0,
    static_leverage=None,
    tp_fee_type=TakeProfitFeeType.Limit,
    tp_strategy_type=TakeProfitStrategyType.RiskReward,
    trail_sl_bool=True,
    z_or_e_type=None,
)

In [ ]:
dos_cart_arrays = dos_cart_product(
    dos_arrays=dos_arrays,
)
dynamic_order_settings = nb_get_dos(
    dos_cart_arrays=dos_cart_arrays,
    dos_index=0,
)

In [ ]:
log_func_type = types.void(types.unicode_type)
log_func_list = typed.List.empty_list(log_func_type.as_type())

str_func_type = types.unicode_type(types.float64)
str_func_list = typed.List.empty_list(str_func_type.as_type())

# log_func_list.append(nb_logger_pass)
# log_func_list.append(nb_logger_pass)
# log_func_list.append(nb_logger_pass)
# log_func_list.append(nb_logger_pass)

# str_func_list.append(nb_stringer_pass)
# str_func_list.append(nb_stringer_pass)
# str_func_list.append(nb_stringer_pass)
# str_func_list.append(nb_stringer_pass)
# str_func_list.append(nb_stringer_pass)

log_func_list.append(nb_log_debug)
log_func_list.append(nb_log_info)
log_func_list.append(nb_log_warning)
log_func_list.append(nb_log_error)

str_func_list.append(nb_float_to_str)
str_func_list.append(nb_log_datetime)
str_func_list.append(nb_candle_body_str)
str_func_list.append(nb_z_or_e_str)
str_func_list.append(nb_os_to_str)

In [ ]:
nb_sl_based_on_candle_body(
    bar_index=60,
    candles=candles,
    logger=log_func_list,
    price_tick_step=1,
    sl_based_on_add_pct=0.0005,
    sl_based_on_lookback=20,
    sl_bcb_price_getter=nb_max_price_getter,
    sl_bcb_type=CandleBodyType.High,
    sl_price_calc=nb_short_sl_price_calc,
    stringer=str_func_list,
)